# **Imports**🎇

In [26]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from tqdm import tqdm
import os
import json
from PIL import Image
import warnings
warnings.filterwarnings("ignore")

import torch
from torch.utils.data import DataLoader, random_split, Dataset
import torch.optim as optim

import torchvision
from torchvision import transforms as T
from torchvision.utils import make_grid
from torchvision.transforms import functional as F

# **Utils**

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

n_epoch = 50

# **Prepare Labels and Files**

In [9]:
# !labelme

In [10]:
# for folder in ['train', 'valid', 'test']:
#     for file in os.listdir(os.path.join(folder, 'photo/')):
#         filename = file.split('.')[0]+'.json'
#         existing_filepath = os.path.join('labels', filename)
#         if os.path.exists(existing_filepath):
#             new_filepath = os.path.join(folder, 'label', filename)
#             os.replace(existing_filepath, new_filepath)


# **Functions**

In [68]:
class TrumpDataset(Dataset):
    def __init__(self, phase):
        self.phase = phase

        self.images_list = []
        self.labels = []
        for item in os.listdir(os.path.join(phase, 'photo')):
            img_path = os.path.join(phase, 'photo', item)
            self.images_list.append(img_path)
            self.labels.append(os.path.join(phase, 'label', '.'.join(item.split('.')[:-1]) + '.json'))


    def __getitem__(self, item):
        img_path = os.path.join(self.images_list[item])
        img = Image.open(img_path).convert('RGB')
        img = F.to_tensor(img)
          
        with open(self.labels[item]) as f:
            loal = json.load(f)
            points = []
            for i in range(len(load['shapes'])):
                points.append(
                    [load['shapes'][i]['points'][0][0],
                     load['shapes'][i]['points'][0][1],
                     load['shapes'][i]['points'][1][0],
                     load['shapes'][i]['points'][1][1]])

        boxes = torch.tensor(points, dtype=torch.float32)
        labels = torch.ones((len(load['shapes']), ), dtype=torch.int64)
        #
        target = {}
        target['boxes'] = boxes
        target['labels'] = labels
        #
        return img, target

    def __len__(self):
        return len(self.images_list)